In [1]:
import tensorflow as tf
import statistics
# import tensorflow_probability as tfp
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../combined data.csv')
df

,game_id,Datetime,Team1 ID,Team2 ID,Team1 Score,Team2 Score,Point Differential,Team1 Home,Team2 Home,Team1 team_id,...,Team2 KenPom Def Eff,Team2 KenPom AdjTempo,Team2 KenPom SOS,Team2 KenPom Off SOS,Team2 KenPom Def SOS,Team2 BT Eff Height,Team2 BT WAB,Team2 BT Power,Team2 BT Experience,Team2 BT Talent
0,514736,11/14/2014 0:00,Alabama A&M_2015,Dayton_2015,52,76,-24,0,1,Alabama A&M_2015,...,95.0,63.4,3.39,105.4,102.0,78.250,0.938,0.798,1.540,46.592
1,514738,11/14/2014 0:00,Alcorn St._2015,California_2015,57,91,-34,0,1,Alcorn St._2015,...,99.0,65.3,5.58,107.8,102.2,80.886,-3.497,0.641,1.637,52.204
2,514740,11/14/2014 0:00,American_2015,Temple_2015,37,40,-3,0,1,American_2015,...,94.2,65.4,2.21,104.8,102.6,80.368,-0.276,0.794,1.941,49.622
3,514742,11/14/2014 0:00,Ball St._2015,Utah_2015,72,90,-18,0,1,Ball St._2015,...,89.2,60.5,8.19,108.8,100.6,81.657,3.720,0.945,1.554,27.543
4,514743,11/14/2014 0:00,Charleston Southern_2015,Mississippi_2015,66,65,1,0,1,Charleston Southern_2015,...,100.6,66.2,7.10,107.3,100.2,80.963,-0.661,0.787,2.280,46.963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52033,5254271,4/2/2024 0:00,Utah_2024,Indiana St._2024,90,100,-10,0,0,Utah_2024,...,103.2,70.9,3.07,107.7,104.7,80.221,1.928,0.852,2.079,0.200
52034,5254269,4/4/2024 0:00,Indiana St._2024,Seton Hall_2024,77,79,-2,0,0,Indiana St._2024,...,98.1,66.8,10.10,111.8,101.7,79.886,0.943,0.856,2.484,62.979
52035,5254134,4/6/2024 0:00,Alabama_2024,Connecticut_2024,72,86,-14,0,0,Alabama_2024,...,91.1,64.6,12.42,113.2,100.8,81.643,11.210,0.980,1.710,58.255
52036,5254135,4/6/2024 0:00,N.C. State_2024,Purdue_2024,50,63,-13,0,0,N.C. State_2024,...,94.6,67.0,14.65,114.4,99.8,83.272,10.620,0.965,1.854,52.251


In [3]:
df = df.drop(columns=["game_id", "Datetime", "Team1 ID", "Team2 ID", "Team1 Score", "Team2 Score", "Team1 team_id", "Team1 Team1 team_name", "Team1 year", "Team2 team_id", "Team2 team_name", "Team2 year"])
df = df.dropna()
df.dtypes

Point Differential       int64
Team1 Home               int64
Team2 Home               int64
Team1 Pts/Gm           float64
Team1 Opp. Pts/Gm      float64
                        ...   
Team2 BT Eff Height    float64
Team2 BT WAB           float64
Team2 BT Power         float64
Team2 BT Experience    float64
Team2 BT Talent        float64
Length: 103, dtype: object

In [4]:
train_df = df.sample(frac = 0.7)
test_df = df.drop(train_df.index)



# mean_val = tf.reduce_mean(train_df)
# std_dev = np.std(train_df)
# normalized_train = (train_df - mean_val) / std_dev

# normalized_test = (test_df - mean_val) / std_dev
# min_val = train_df.min(axis = 0)
# max_val = train_df.max(axis = 0)
# val_range = max_val - min_val
# train_df = (train_df - min_val) / val_range
# test_df = (test_df - min_val) / val_range

y_train = train_df['Point Differential']
y_test = test_df['Point Differential']

x_train = train_df.drop('Point Differential', axis = 1)
x_test = test_df.drop('Point Differential', axis = 1)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(x_train))

In [5]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

       Team1 Home  Team2 Home  Team1 Pts/Gm  Team1 Opp. Pts/Gm  Team1 FG%  \
5773            0           0          67.6               69.7       39.1   
14153           0           1          68.8               75.2       39.5   
6805            0           1          66.1               77.5       41.4   
7873            0           1          79.5               72.3       43.5   
8910            0           1          78.9               78.3       44.2   
...           ...         ...           ...                ...        ...   
44789           0           1          68.3               75.1       43.0   
41289           0           1          70.0               62.8       45.8   
11552           0           0          76.8               81.0       45.7   
23546           0           1          68.2               70.7       40.9   
2232            0           1          78.3               69.5       46.5   

       Team1 Opp. FG%  Team1 2Pt%  Team1 Opp. 2Pt%  Team1 3Pt%  \
5773     

In [6]:
input_shape = [x_train.shape[1]]

In [7]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = input_shape),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = 1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 102)              205       
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                6592      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,022
Trainable params: 10,817
Non-trainable params: 205
_________________________________________________________________


In [8]:
model.compile(optimizer =tf.keras.optimizers.Nadam(learning_rate=0.00006), loss = 'mse')
losses = model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 256, epochs=15)

Epoch 1/15
134/134 [==============================] - 2s 7ms/step - loss: 214.0559 - val_loss: 199.7396
Epoch 2/15
134/134 [==============================] - 1s 6ms/step - loss: 189.9401 - val_loss: 171.7801
Epoch 3/15
134/134 [==============================] - 1s 5ms/step - loss: 159.7377 - val_loss: 141.7907
Epoch 4/15
134/134 [==============================] - 1s 5ms/step - loss: 134.0196 - val_loss: 122.3990
Epoch 5/15
134/134 [==============================] - 1s 5ms/step - loss: 120.9252 - val_loss: 115.1469
Epoch 6/15
134/134 [==============================] - 1s 5ms/step - loss: 116.4352 - val_loss: 112.8078
Epoch 7/15
134/134 [==============================] - 1s 5ms/step - loss: 114.6430 - val_loss: 111.6278
Epoch 8/15
134/134 [==============================] - 1s 5ms/step - loss: 113.5327 - val_loss: 110.8121
Epoch 9/15
134/134 [==============================] - 1s 5ms/step - loss: 112.7288 - val_loss: 110.1864
Epoch 10/15
134/134 [==============================] - 1s 5ms/st

In [9]:
model.predict(x_test.iloc[0:9, :])

1/1 [==============================] - 0s 130ms/step


array([[-23.910986 ],
       [-14.563387 ],
       [-14.907513 ],
       [-23.191916 ],
       [ -4.8469462],
       [ -0.8570383],
       [ -2.5706596],
       [  3.302521 ],
       [  4.0184846]], dtype=float32)

In [10]:
y_test.iloc[0:9]

1    -34
4      1
6    -13
13   -36
15    -1
20   -11
23    -6
25    -7
27   -30
Name: Point Differential, dtype: int64